# Audio Processing

This notebook plays with different audio procesing algorithms and develops a flexible way to wire them up.

The problem: ad-hoc implementations of processing algorithms mix I/O and algorithm logic, making the algorithm less portable.

The solution: define standard patterns for algorithms to declare inputs and outputs and specify how data moves from inputs to outputs, and for clients to create and join algorithms into a full processing graph.

Use cases:
- Recording a song
- Creating a visualization that can analyze audio files or live inputs
- Creating a real-time audio effect
- Creating an offline audio effect

# Design

The data flow building block is the Channel, a one-way 1-N data transport. 

Channel output can be connected to buffers or transform methods. Channels have a write method for input.

Algorithms use one channel per logical input and output. They configure the output side of their input channels, input side of their output channels using buffers or transform objects. They are also responsible for scheduling work to read and write any buffers used to connect inputs and outputs.

In [14]:
%matplotlib widget

import pyaudio
import numpy as np
from IPython.display import display
import ipywidgets as widgets
from pprint import pprint
import logging
import threading
import librosa
import time

from jupyter_utils import *
from audio_tools import *

# Set up the root logger and display its output in this cell.
logger = get_cell_logger(__name__)
logger.show_logs()

Button(description='Clear logs', style=ButtonStyle())

Output(layout=Layout(border='1px solid white', height='300px', overflow='auto'))

In [ ]:
graph_running = False

def run_graph(file):
    # Create graph nodes
    gain = Gain()
    audio = AudioFileLoader()
    input_buffer = AudioBuffer(num_frames=10**6)
    output_queue = AudioQueue(num_frames=4096)

    # Connect nodes
    audio.output.add_sink(gain.input)
    audio.output.add_sink(input_buffer)
    gain.output.add_sink(output_queue)

    # Start data flow
    audio.load_async(file)

    def read_output():
        while graph_running:
            time.sleep(.1)
            data = output_queue.read(output_queue.read_available())
            logger.info(f'read {data.shape[0]} frames, audio.is_loading {audio.is_loading()}, input_buffer read {input_buffer.read_available()}')

    global graph_running
    graph_running = True
    threading.Thread(target=read_output).start()

debug_view = widgets.Output(layout={'border': '1px solid white'})
    
@debug_view.capture(clear_output=True)
def toggle(b):
    global graph_running
    if graph_running:
        graph_running = False
        run_button.description = "Start"
    else:
        run_button.description = "Stop"
        run_graph('a2_mp_rr2.wav')

run_button = widgets.Button(description="Start")
run_button.on_click(toggle)
display(run_button)
logger.show_logs()
display(debug_view)


In [4]:
try:
    daw.terminate()
except NameError:
    pass

daw = Daw()

Button(description='Start', style=ButtonStyle())

Button(description='Terminate', style=ButtonStyle())

Button(description='Clear logs', style=ButtonStyle())

Output(layout=Layout(border='1px solid white', height='300px', overflow='auto'))

Output(layout=Layout(border='1px solid white'))